Установка библиотек

В колабе возможно все уже есть, но лучше все равно установить

In [ ]:
!apt-get install unzip wget -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
wget is already the newest version (1.19.4-1ubuntu2.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!pip install transformers
!pip install -U sentence-transformers
!pip3 install torch torchvision
!pip install pymorphy2

     |████████████████████████████████| 2.1MB 20.3MB/s 
     |████████████████████████████████| 3.3MB 53.1MB/s 
     |████████████████████████████████| 901kB 49.9MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 1.2MB 13.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=29e99aa3f26f1dfd5342a022ba31fcd7239dedd7a79dd6bc8bc60229d26e165c
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 8.2MB 21.1MB/s 


In [ ]:
!pip3 install nltk

In [ ]:
import torch
from torch.utils.data import TensorDataset, random_split
from torchtext import data
from transformers import AutoTokenizer, AutoModel

In [ ]:
!unzip -o '/content/drive/MyDrive/Colab Notebooks/Distillation/test-lenta.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/Distillation/test-lenta.zip
  inflating: test-lenta.csv          


# Данные


Подгружаем фильтрованный датасет test-lenta.csv

In [ ]:
import pandas as pd

dataset = pd.read_csv('test-lenta.csv')
dataset.head()

,Unnamed: 0,topic,title
0,0,Из жизни,Изнывающего от жары олененка застали за «намат...
1,1,Россия,РБК: Абхазия официально просит у России защиты...
2,2,Наука и техника,Россия построит подводный флот Вьетнама
3,3,Россия,Большинство россиян считает операцию в Беслане...
4,4,Культура,"""Одержимость"" возглавила рейтинг самых кассовы..."


In [ ]:
# Подгружаем SBERT-модель и SBERT-токенайзер 
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [ ]:
import pymorphy2
import nltk
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize,RegexpTokenizer
nltk.download('stopwords')
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

delimer = RegexpTokenizer(r'\w+')

# лемматизация входного слова
def lemmatize(word):
  p = morph.parse(word)[0]
  p = p.normal_form
  return p

# удаление стоп-слов
def get_without_stopwords(s):

  words = delimer.tokenize(s)
  without_stop_words = [lemmatize(word) for word in words if not word in stop_words]
  return ' '.join(without_stop_words)

# приведение к нижнему регистру
def to_lower(sen):
  return sen.lower()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# конвертация к формату torch.tensor
def to_dataset(x, y):
    torch_x = torch.tensor(x, dtype=torch.long)
    torch_y = torch.tensor(y, dtype=torch.float)
    return TensorDataset(torch_x, torch_y)

In [ ]:
# изменение устройства
def device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")


def to_device(text, real_label):
    text = text.to(device())
    real_label = real_label.to(device())
    return text, real_label


In [ ]:
# функция подсчета количества весов у модели
def get_weights(model):
  number = 0
  for key in model.state_dict():
    number += model.state_dict()[key].shape[0]
  return number

In [ ]:
from torch import nn
from torch.autograd import Variable


# Модель-ученик
class SimpleLSTM(nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, batch_size, device=None):
        super(SimpleLSTM, self).__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.device = self.init_device(device)
        self.hidden = self.init_hidden()

    @staticmethod
    def init_device(device):
        if device is None:
            return torch.device('cuda')
        return device

    def init_hidden(self):
        return (Variable(torch.zeros(2 * self.n_layers, self.batch_size, self.hidden_dim).to(self.device)),
                Variable(torch.zeros(2 * self.n_layers, self.batch_size, self.hidden_dim).to(self.device)))

    def forward(self, text, text_lengths=None):
        self.hidden = self.init_hidden()
        x = self.embedding(text)
        x, self.hidden = self.rnn(x, self.hidden)
        hidden, cell = self.hidden
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        x = self.fc(hidden)

        return x

In [ ]:
def get_optimizer(model):
    optimizer = torch.optim.Adam(model.parameters())
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 2, gamma=0.9)
    return optimizer, scheduler


def epoch_train_func(model, dataset, loss_func, batch_size):
    train_loss = 0
    train_sampler = RandomSampler(dataset)
    data_loader = DataLoader(dataset, sampler=train_sampler,
                             batch_size=batch_size,
                             drop_last=True)
    model.train()
    optimizer, scheduler = get_optimizer(model)
    for i, (tokens, embedding) in enumerate(tqdm(data_loader, desc='Train')):
        tokens, embedding = to_device(tokens, embedding)
        model.zero_grad()
        output = model(tokens.t(), None).squeeze(1)
        loss = loss_func(output, embedding)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    scheduler.step()
    return train_loss / len(data_loader)

def epoch_evaluate_func(model, eval_dataset, loss_func, batch_size):
    eval_sampler = SequentialSampler(eval_dataset)
    data_loader = DataLoader(eval_dataset, sampler=eval_sampler,
                             batch_size=batch_size,
                             drop_last=True)

    eval_loss = 0.0
    model.eval()
    for i, (tokens, embedding) in enumerate(tqdm(data_loader, desc='Val')):
        tokens, embedding = to_device(tokens, embedding)
        output = model(tokens.t(), None).squeeze(1)
        loss = loss_func(output, embedding)
        eval_loss += loss.item()

    return eval_loss / len(data_loader)

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import os


class LSTMBaseline(object):
    vocab_name = 'distillation_model.pt'
    weights_name = 'simple_lstm.pt'

    def __init__(self, settings):
        self.settings = settings
        self.criterion = torch.nn.MSELoss().to(device())

    def loss(self, student_embedding, teacher_embedding):
        return self.criterion(student_embedding, teacher_embedding)

    def model(self, text_field):
        model = SimpleLSTM(
            input_dim=len(text_field.vocab),
            embedding_dim=50,
            hidden_dim=256,
            output_dim=1024,
            n_layers=2,
            bidirectional=True,
            dropout=0.2,
            batch_size=self.settings['train_batch_size'])
        return model

    def train(self, X, y, output_dir, tokenizer):        
        dataset = to_dataset(X, y)
        val_len = int(len(dataset) * 0.1)
        train_dataset, val_dataset = random_split(dataset, (len(dataset) - val_len, val_len))

        model = self.model(tokenizer)
        model.to(device())

        self.full_train(model, train_dataset, val_dataset, output_dir)

    def show_training(self, loss_values, val_loss_values):

        epochs = range(1, settings['num_train_epochs'] + 1)

        plt.plot(epochs, loss_values, 'bo', label='Training loss')
        plt.plot(epochs, val_loss_values, 'b', label='Validation loss')

        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        plt.show()

    def full_train(self, model, train_dataset, val_dataset, output_dir):

        train_loss_list = []
        eval_loss_list = []

        train_settings = self.settings
        num_train_epochs = train_settings['num_train_epochs']
        best_eval_loss = None
        for epoch in range(num_train_epochs):
            train_loss = epoch_train_func(model, train_dataset, self.loss, self.settings['train_batch_size'])
            eval_loss = epoch_evaluate_func(model, val_dataset, self.loss, self.settings['eval_batch_size'])

            train_loss_list.append(train_loss)
            eval_loss_list.append(eval_loss)

            if best_eval_loss == None or eval_loss < best_eval_loss:
                best_eval_loss = eval_loss
                torch.save(model.state_dict(), os.path.join(output_dir, self.weights_name))

        self.show_training(train_loss_list, eval_loss_list)
        

In [ ]:
# количество предложений в выборке
DATASET_SIZE = 1000

In [ ]:
from tqdm import tqdm
max_len = 20
X = list(dataset['title'][:DATASET_SIZE].apply(to_lower).apply(get_without_stopwords))

X_encoded = tokenizer(X, padding=True, truncation=True, max_length=max_len, return_tensors='pt')

In [ ]:
with torch.no_grad():
    model_output = model(**X_encoded)
    sentence_embeddings = model_output[0][:,0] #Take the first token ([CLS]) from each sentence 

In [ ]:
settings = {
    'train_batch_size' : 32,
    'num_train_epochs' : 6,
    'eval_batch_size': 32,

}

In [ ]:
baseline = LSTMBaseline(settings)

In [ ]:
baseline.train(X_encoded['input_ids'], sentence_embeddings, '/content/drive/MyDrive/Colab Notebooks/Distillation', tokenizer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


NameError: ignored